# Mushroom classification
In this notebook, we will look into the mushroom dataset to analyze its characteristics and gain insights. Following our exploration, we will develop a predictive model to determine the edibility of mushrooms using the available variables, which are these:
* cap-diameter - diameter of the cap of the mushroom (in cm)
* cap-shape - shape of the cap of the mushroom bell=1, conical=2, convex=3, flat=4,
sunken=5, spherical=6, others=7

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import statsmodels.api as sm
from patsy import dmatrices

df = pd.read_csv('../Data/MushroomDataset/secondary_data.csv', sep=';')



df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   class                 61069 non-null  object 
 1   cap-diameter          61069 non-null  float64
 2   cap-shape             61069 non-null  object 
 3   cap-surface           46949 non-null  object 
 4   cap-color             61069 non-null  object 
 5   does-bruise-or-bleed  61069 non-null  object 
 6   gill-attachment       51185 non-null  object 
 7   gill-spacing          36006 non-null  object 
 8   gill-color            61069 non-null  object 
 9   stem-height           61069 non-null  float64
 10  stem-width            61069 non-null  float64
 11  stem-root             9531 non-null   object 
 12  stem-surface          22945 non-null  object 
 13  stem-color            61069 non-null  object 
 14  veil-type             3177 non-null   object 
 15  veil-color         